# ASTM13 Dynamical astronomy

## P1: Oort's constants

### Purpose

The purpose of this project is to estimate Oort's constants $A$ and $B$ from proper motion data in the Gaia catalogue.

Oort's constants are related to the rotation curve $V(R)$ of the Galaxy through Eq. (4.14-15) from the Lecture Notes:
\begin{equation}
V_0/R_0 = A-B\quad(4.14) \\
\left(\frac{\text{d}V}{\text{d}R}\right)_0 = -(A+B)\quad(4.15)
\end{equation}
where the index $0$ stands for quantities at the Sun's distance from the galactic centre. Thus $A$ and $B$ give information about the rotation curve within a few kiloparsecs from the Sun, where $V(R)$ is approximately linear.

This notebook begins with a very brief theory section followed by some useful Python code. The instructions for the report conclude the notebook.

### Theory

Oort's formulae for the galactic rotation are derived in Chapter 4 of the Lecture Notes. The relevant formula is that for the proper motion in galactic longitude:
\begin{equation}\label{e1}
A\cos 2\ell + B = K\mu_\ell\quad(4.13)
\end{equation}
where $K=4.7405$ is the conversion factor from mas$\,$yr$^{-1}$ to km$\,$s$^{-1}\,$kpc$^{-1}$ and $\mu_\ell$ is the proper motion in longitude.

The proper motions of nearby stars are dominated by the scatter of the individual velocities of the stars, including that of the Sun. As we will see in the next project, these velocities are typically of the order of 20 km$\,$s$^{-1}$. The proper motion $\mu$ (in mas$\,$yr$^{-1}$) is related to the tangential velocity $\tau$ (in km$\,$s$^{-1}$) and distance $r$ (in kpc) by
\begin{equation}\label{e2}
\mu = \tau/(Kr)
\end{equation}
so $\tau\simeq 20$ km$\,$s$^{-1}$ at 0.1 kpc distance corresponds to $\mu\simeq 40$ mas$\,$yr$^{-1}$, while at 1 kpc it corresponds to 4 mas$\,$yr$^{-1}$. On the other hand, the amplitude of the effect in proper motion due to the differential rotation of the Galaxy is $A/K\simeq 3$ mas$\,$yr$^{-1}$ according to the lecture notes. Thus, the peculiar velocities of the stars will dominate up to distances of about 1 kpc. For more distant stars the proper motions are dominated by the differential rotation. To get a good result from Eq. (4.13) we should therefore select distant stars. However, more nearby stars could still give a sensible result as the peculiar motions tend to average out in a sufficiently large sample.

### Python help

We must first obtain the stellar sample. The code below should be mostly familiar from Project 0.

In [ ]:
import numpy as np
import astropy.units as u

from astropy.table import Table, QTable, Column
from os import listdir, rename
from matplotlib import pyplot as plt


def perform_Gaia_query(G_lim):
    """Perform a query from Gaia DR2.

    The query will include all the stars brighter with G<G_lim that also have a 5-parameter
    astrometric solution together with BP and RP measurements. Their astrometric parameters,
    uncertainties and correlations together with G, BP and RP magnitudes are saved to a VOTable.
    The name of the VOTable file is returned.
    """
    from astroquery.gaia import Gaia

    astrometry = ('ra', 'dec', 'parallax', 'pmra', 'pmdec')
    columns_to_query = ''
    # Add astrometric measurements, uncertanties and correlations to the columns to be queried
    for i, col in enumerate(astrometry):
        columns_to_query += col+', '
        columns_to_query += col+'_error, '
        for col2 in astrometry[i+1:]:
            columns_to_query += col+'_'+col2+'_corr, '
    # Make sure the magnitudes in the three bands are queried too
    for band in ('g', 'bp', 'rp'):
        columns_to_query += 'phot_'+band+'_mean_mag, '
    # Remove the last comma
    columns_to_query = columns_to_query[:-2]
    job = Gaia.launch_job_async(f'SELECT {columns_to_query}\
                                 FROM gaiadr2.gaia_source\
                                 WHERE phot_g_mean_mag < {G_lim}\
                                     AND parallax IS NOT NULL\
                                     AND bp_rp IS NOT NULL\
                                 ;', dump_to_file=True)
    return job.outputFile


G_lim = 11
filename = f'brighter_than_{G_lim}.vot'
if filename not in listdir():
    print('Performing a query from Gaia archive. This might take a while...')
    rename(perform_Gaia_query(G_lim), filename)
else:
    print('Reading data from a stored file.')
data = QTable.read(filename)
# Astropy QTable has problems parsing the proper motion units, so we have to help it along.
for col in data.columns:
    if 'pm' in col and 'corr' not in col:
        data[col] = np.array(data[col])*u.Unit(str(data[col].unit))
print('Done.')

We now need to convert the astrometry from equatorial coordinates to the galactic system. Coordinate conversion could be done very easily using Astropy, as was demonstrated in Project 0, but to ensure that the uncertainties are also converted we will use the module ``coordTransform.py`` instead. Somewhat unfortunately that module is meant to work with NumPy arrays rather than Astropy tables, but the code below takes care of the necessary conversions.

In [ ]:
def construct_covariance_matrix(stars):
    """Create the covariance matrix of the astrometric parameters of input stars."""
    covariance = np.ones((len(stars), 5, 5))
    astrometry = ('ra', 'dec', 'parallax', 'pmra', 'pmdec')
    for i, col in enumerate(astrometry):
        # This covariance matrix is set up as a numpy array, so we need to strip the units.
        covariance[:, i, :] *= np.array(stars[col+'_error'])[:, np.newaxis]
        covariance[:, :, i] *= np.array(stars[col+'_error'])[:, np.newaxis]
        for j in range(i+1, 5):
            covariance[:, i, j] *= np.array(stars[col+'_'+astrometry[j]+'_corr'])
            covariance[:, j, i] = covariance[:, i, j]
    return covariance


def convert_icrs_to_gal(stars):
    """Convert coordinates from ICRS to the Galactic system."""
    from coordTransform import transformIcrsToGal
    gal_params = ('l', 'b', 'pml', 'pmb')
    coords = np.array(stars['ra', 'dec', 'parallax', 'pmra', 'pmdec']).view((float, 5))
    covariance = construct_covariance_matrix(stars)
    gal_coords, gal_uncerts, gal_covariance = transformIcrsToGal(coords, 0, EqC=covariance)
    temp = QTable(gal_coords[:, np.array((0, 1, 3, 4))], names=gal_params)
    stars.add_columns(temp.columns.values())
    for i, name, unit, err_unit in zip((0, 1, 3, 4), gal_params,
                                       (u.deg, u.deg, u.mas/u.yr, u.mas/u.yr),
                                       (u.mas, u.mas, u.mas/u.yr, u.mas/u.yr)):
        stars[name] *= unit
        stars.add_column(Column(np.sqrt(gal_covariance[:, i, i])), name=name+'_error')
        # Right ascension and declination are in deg, but their uncertanties in mas.
        # We must ensure the same for galactic longitude and latitude.
        stars[name+'_error'] *= err_unit
    return stars


# Don't compute the galactic coordinates if they are already computed.
if 'b' not in data.columns:
    data = convert_icrs_to_gal(data)

The plot below should be familiar from Project 0.

In [ ]:
close_to_midplane = np.abs(data['b']) < 6.4*u.deg
plt.scatter(data[close_to_midplane]['ra'], data[close_to_midplane]['dec'])
plt.title('Equatorial coordinates of stars within 6.4 degrees from the Galactic midplane')
plt.xlabel(rf'$\alpha$ [{data["ra"].unit.to_string("latex")}]')
plt.ylabel(rf'$\delta$ [{data["dec"].unit.to_string("latex")}]')
plt.show()

It might be useful to comment briefly on the factor $K$ present in Eq. (4.13). Normally unit conversion is not written out explicitly. For example, declinations of stars are often written simply as $\delta$ with the unit left unspecified and it would be quite unusual to see it explicitly written out that $d=L\delta$ where $d$ is the declination in radians, $\delta$ the declination in degrees and $L=\pi/180$ the conversion factor from degrees to radians. However explicitly writing out the conversion of mas$\,$yr$^{-1}$ to km$\,$s$^{-1}\,$kpc$^{-1}$ is indeed a common enough practise in literature. When using Astropy it is not a good idea to try to implement the unit conversion in the quirky way using $K$ as it has been done historically. You should perform the unit conversion with an equivalency instead, as demonstrated below.

In [ ]:
mu = 1*u.mas/u.yr
print(f'{mu} is {mu.to(u.km/u.s/u.kpc, equivalencies=u.dimensionless_angles()):.4f}.')

In [ ]:
data.show_in_notebook(display_length=5)

With the help of the code above you should now be able to address the following in your report.

### Report instructions

#### Stellar selection

Points to consider (and to answer in the report):
* How should the stars be selected with respect to their parallaxes? Remember that most parallaxes have random errors that are of the order of 0.04$\,$mas.
* What can be considered "small $|b|$" in this context?
* What might be a suitable colour interval in this case?

#### Calculations

Based on Eq. (4.13) and the given data $(\ell,\mu_\ell)$, use Python to make a least-squares estimate of $A$ and $B$. One possible method  is to make a linear regression of $\mu_\ell$ versus $\cos 2\ell$. Plot $\mu_\ell$ versus $\ell$ for the stars in the sample and plot also the fitted cosine curve.

Report the estimated values of $A$ and $B$. Try also to estimate the uncertainty in the resulting values. (How can that be done?)

Repeat the whole process for stars in different colour ranges. Do they give consistent results? What happens if more nearby stars are included? Are the variations between different colour groups consistent with the estimated uncertainties? Which colours give the most reliable determination? Why? How do the estimates compare to the "textbook values"?

What are your best estimate of $A$ and $B$ (with uncertainties)?

Can you think of any improvement of the method?

#### Writing

The report should describe the purpose, data (including precise selection criteria), method, results, plots, and a discussion of the results guided by the questions above. The code you used for producing the results should be included.

In [ ]:
#changable parameters

K = 4.7405 #Conversion factor
E = 10 #magnitude of parallax error
h = 0.3 #scale height
kpc_min = 0.9 #lower limit of distance from the sun
kpc_max = 1 #max limit of the distance from the sun
col_min = 0 #minimum colour used
col_max = 1 #maximum -||-
step_col = 0.2 #color step size
mag_lim = 11 #limiting magnitude

G = [(data['parallax'].value > E*data['parallax_error'].value) & #parallax vs error
       (1/data['parallax'].value > kpc_min) & (1/data['parallax'].value < kpc_max) & #Limting distance
       (abs(data['b'].value)< np.rad2deg(np.arctan(h/kpc_max))) & #Limiting longitude
       ((data['phot_bp_mean_mag'].value - data['phot_rp_mean_mag'].value) > i) & #Color ranges
       ((data['phot_bp_mean_mag'].value - data['phot_rp_mean_mag'].value) < i+0.2) &
       (data['phot_g_mean_mag'].value < mag_lim) for i in np.arange(col_min,col_max,step_col)] #Magnitude lim

selections = [data[G[i]] for i in range(5)] #Using the filtered data

#for colors < 0
G_col_less_than_0 = (data['parallax'].value > E*data['parallax_error'].value) & (1/data['parallax'].value > kpc_min) & (1/data['parallax'].value < kpc_max) &  (abs(data['b'].value)< np.rad2deg(np.arctan(h/kpc_max))) &((data['phot_bp_mean_mag'].value - data['phot_rp_mean_mag'].value) < 0) & (data['phot_g_mean_mag'].value < mag_lim)
                
selection_col_less_than_0 = data[G_col_less_than_0]#Using filtered data for bp-rp < 0

#Parameters colors < 0
cos2l_col_0 = np.cos(np.deg2rad(2*selection_col_less_than_0['l'].value))
mu_l_col_0 = selection_col_less_than_0['pml'].value
l_col_0 = selection_col_less_than_0['l'].value

#Fitting and Oorts constant for color < 0
polys_col_0, cov = np.polyfit(cos2l_col_0, K*mu_l_col_0,1,cov = True)
A_col_0 = np.around(polys_col_0[0],1)
B_col_0 = np.around(polys_col_0[1],1)

#Coviarance color < 0 for error estimation
covariance_col_0 = np.around(np.sqrt(np.diag(cov)),1)

#Parameters color > 0
cos2l = [np.cos(np.deg2rad(2*selections[i]['l'].value)) for i in range(5)]
mu_l_new = [selections[i]['pml'].value for i in range(5)]
l_new = [selections[i]['l'].value for i in range(5)]

#Fitting and finding Oorts constants 
polys =[np.polyfit(cos2l[i],K*mu_l_new[i],1,cov = True) for i in range(5)]

#Calculate the oort's constants for color ranges 0 < bp-rp < 1 and round to first decimal
A = [np.around(polys[i][0][0],1) for i in range(5)]
B = [np.around(polys[i][0][1],1) for i in range(5)]


#Covariances for error estimation
covariances = [np.around(np.sqrt(np.diag(polys[i][1])),1) for i in range(5)]

def Oort1(l_1,mu_l):
    '''This describes the proper motion as a function of cos(2l), where l is the galactic longitude
    with Oort's constants as A and B'''
    mu_l_1 = [((A[i]*np.cos(np.deg2rad(2*l_1)) + B[i]) / K) for i in range(5)]
    return mu_l_1

#Using the function when plotting and make data fit
pp = [Oort1(np.sort(l_new[i]), mu_l_new[i])[1] for i in range(5)]
pp_col_0 = Oort1(np.sort(l_col_0), mu_l_col_0)

#Defining axes and plot properties
fig, ax = plt.subplots(3, 2,figsize=(20,28))
fig.text(0.5, 0.1,r'$\ell$ [deg]', ha='center', va='center', fontsize = 25)
fig.text(0.08, 0.5,r'$\mu_{\rm{\ell}}$ [mas/yr]', ha='center', va='center', rotation='vertical', fontsize = 25)
label_size = 18
plt.rcParams['xtick.labelsize'] = label_size
plt.rcParams['ytick.labelsize'] = label_size
markersize = 0.5

#Make titles update depending in the chosen parameters
titlelist = []
for i in np.arange(0,1,0.2):
    title = r'$G$ < {} & bp-rp $\in$ ({},{}) & $r$ $\in$ ({},{})'.format(mag_lim, np.around(i,1),np.around(i+0.2,1),kpc_min, kpc_max)
    titlelist.append(title)
title_col_0 = r'$G$ < {} & bp-rp < 0 & $r \in$ ({},{})'.format(mag_lim, kpc_min, kpc_max)

#All of the plots. Plots, titles and legends update automatically when changing parameters 
ax[0,0].plot(l_col_0, mu_l_col_0,'o', ms = markersize, label = r'Stars = {}'.format(len(l_col_0)))
ax[0,0].plot(np.sort(l_col_0), pp_col_0[0], label = r"""$A$ = {} $\pm$ {} km/(kpc s)
$B$ = {} $\pm$ {} km/(kpc s)""".format(A_col_0, covariance_col_0[0],B_col_0, covariance_col_0[1]), linewidth = 3)
ax[0,0].set_title(title_col_0, fontsize = 20)
ax[0,0].grid()
ax[0,0].legend(fontsize = 20, loc = 4)

ax[0,1].plot(l_new[0], mu_l_new[0],'o', ms = markersize, label = r'Stars = {}'.format(len(l_new[0])))
ax[0,1].plot(np.sort(l_new[0]), pp[0], label = r"""$A$ = {} $\pm$ {} km/(kpc s)
$B$ = {} $\pm$ {} km/(kpc s) """.format(A[0], covariances[0][0],B[0], covariances[0][1]), linewidth = 3)
ax[0,1].set_title(titlelist[0], fontsize = 20)
ax[0,1].grid()
ax[0,1].legend(fontsize = 20, loc = 4)

ax[1,0].plot(l_new[1], mu_l_new[1],'o', ms = markersize, label = r'Stars = {}'.format(len(l_new[1])))
ax[1,0].plot(np.sort(l_new[1]), pp[1], label = r"""$A$ = {} $\pm$ {} km/(kpc s)
$B$ = {} $\pm$ {} km/(kpc s)""".format(A[1], covariances[1][0],B[1], covariances[1][1]), linewidth = 3)
ax[1,0].set_title(titlelist[1], fontsize = 20)
ax[1,0].grid()
ax[1,0].legend(fontsize = 20, loc = 4)

ax[1,1].plot(l_new[2], mu_l_new[2],'o', ms = markersize, label = r'Stars = {}'.format(len(l_new[2])))
ax[1,1].plot(np.sort(l_new[2]), pp[2], label = r"""$A$ = {} $\pm$ {} km/(kpc s)
$B$ = {} $\pm$ {} km/(kpc s)""".format(A[2], covariances[2][0],B[2], covariances[2][1]), linewidth = 3)
ax[1,1].set_title(titlelist[2], fontsize = 20)
ax[1,1].grid()
ax[1,1].legend(fontsize = 20, loc = 4)

ax[2,0].plot(l_new[3], mu_l_new[3],'o', ms = markersize, label = r'Stars = {}'.format(len(l_new[3])))
ax[2,0].plot(np.sort(l_new[3]), pp[3], label = r"""$A$ = {} $\pm$ {} km/(kpc s)
$B$ = {} $\pm$ {} km/(kpc s)""".format(A[3], covariances[3][0],B[3], covariances[3][1]), linewidth = 3)
ax[2,0].set_title(titlelist[3], fontsize = 20)
ax[2,0].grid()
ax[2,0].legend(fontsize = 20, loc = 4)

ax[2,1].plot(l_new[4], mu_l_new[4],'o', ms = markersize, label = r'Stars = {}'.format(len(l_new[4])))
ax[2,1].plot(np.sort(l_new[4]), pp[4], label = r"""$A$ = {} $\pm$ {} km/(kpc s)
$B$ = {} $\pm$ {} km/(kpc s)""".format(A[4], covariances[4][0],B[0], covariances[4][1]), linewidth = 3)
ax[2,1].set_title(titlelist[4], fontsize = 20)
ax[2,1].grid()
ax[2,1].legend(fontsize = 20, loc = 4)
#plt.savefig('11_09-1.pdf', dpi = 500)